In [4]:
import sys
from PyQt5.QtWidgets import QApplication
from kiwoom_api.api import Kiwoom, DataFeeder, Executor

In [5]:
app = QApplication(sys.argv)

kiwoom = Kiwoom() # Kiwoom 인스턴스 생성
kiwoom.commConnect() # API 접속
feeder = DataFeeder(kiwoom)

#실행후 기다리면 로그인창이 생성됨
#이 주소로 접속한 뒤 OPEN API를 신청 및 설치하고 실행한다.
#Anaconda 실행환경은 32bit이다.
#https://www.kiwoom.com/nkw.templateFrameSet.do?m=m1408000000

2020-06-10 16:30:39.255852 Connection Successful


In [6]:
import pandas as pd

In [23]:
#일봉 데이터 요청

code = "005930" # 삼성전자

# TR요청(request)에 필요한 parameter는 KOAStudio를 참고하시길 바랍니다.
# OPT10004: 주식호가요청
params = {"종목코드": code, "기준일자": "20200602", "수정주가구분" : "1"} #기준일자 -> 불러오는 데이터의 가장 나중날짜
data = feeder.request(trCode="OPT10081", **params)

2020-06-10 16:32:15.886429  commRqData 주식일봉차트조회요청
{'BASC_DT': '2020-06-10',
 'EVENT': 'eventReceiveTrData',
 'REQUEST_NAME': '주식일봉차트조회요청',
 'TIME': '2020-06-10 16:32:19.254424',
 'TR_CODE': 'OPT10081'}


In [24]:
mul_data = data['멀티데이터']

In [25]:
df_data = pd.DataFrame(mul_data)

In [26]:
cols = [ '일자','현재가', '시가', '고가', '저가', '거래량', '거래대금']

In [27]:
df_data = df_data[cols]

In [28]:
df_data['일자'] = pd.to_datetime(df_data['일자'])

In [29]:
df_data.iloc[:, 1:] = df_data.iloc[:, 1:].astype(int)

In [30]:
df_data = df_data.drop(df_data[df_data['거래량'] == 0].index)

In [31]:
df_data = df_data.sort_values(by='일자').reset_index().drop(['index'], axis=1)

In [32]:
df_data

,일자,현재가,시가,고가,저가,거래량,거래대금
0,2017-12-19,51560,51540,52080,51520,11978600,619174
1,2017-12-20,50880,51500,51760,50820,10080550,515676
2,2017-12-21,49140,51000,51060,49100,15624300,777079
3,2017-12-22,49700,49400,49960,49240,11199650,555690
4,2017-12-26,48200,49760,50100,48200,16039850,787685
...,...,...,...,...,...,...,...
592,2020-05-27,49900,48950,50000,48800,19548479,969952
593,2020-05-28,50400,51100,51200,49900,31309318,1585346
594,2020-05-29,50700,50000,50700,49700,27596961,1390446
595,2020-06-01,51200,50800,51200,50600,16949183,864647


In [33]:
percent = df_data.iloc[:, 1:].diff() / df_data.iloc[:, 1:]

In [36]:
percent = pd.concat([df_data['일자'], percent], axis=1)

In [37]:
percent

,일자,현재가,시가,고가,저가,거래량,거래대금
0,2017-12-19,NaN,NaN,NaN,NaN,NaN,NaN
1,2017-12-20,-0.013365,-0.000777,-0.006182,-0.013774,-0.188288,-0.200704
2,2017-12-21,-0.035409,-0.009804,-0.013709,-0.035031,0.354816,0.336392
3,2017-12-22,0.011268,-0.032389,-0.022018,0.002843,-0.395070,-0.398404
4,2017-12-26,-0.031120,0.007235,0.002794,-0.021577,0.301761,0.294528
...,...,...,...,...,...,...,...
592,2020-05-27,0.013026,0.005107,0.011000,0.004098,0.226155,0.234144
593,2020-05-28,0.009921,0.042074,0.023438,0.022044,0.375634,0.388176
594,2020-05-29,0.005917,-0.022000,-0.009862,-0.004024,-0.134521,-0.140171
595,2020-06-01,0.009766,0.015748,0.009766,0.017787,-0.628218,-0.608108
